In [1]:
import pandas as pd
import numpy as np
import csv
import sys
import getopt
from Bio import SeqIO
import re
import os

In [2]:
# peptide length 8-11, removing gibbs junk and DB matched
data = pd.read_csv('CSV\\1130_MHCI_1.csv')
gibbs = pd.read_csv('CSV\\1130_MHCI_1_Gibbs.csv')
data = data[data["Found By"] != 'DB Search']
gibbs = gibbs[gibbs["Gn"].isin([0,2,3])]
gibbs = gibbs[gibbs["Sequence"].str.len().between(8,11)]
data = data[data["Peptide"].isin(gibbs["Sequence"])]
#data.to_csv('CSV\\non_can.csv')
list = data["Peptide"] # list of peptides
pep = list.tolist()

In [66]:
#fn for creating files from a list and file_name
def create_files(pep, file_name):

    ofile = open(file_name + ".fasta", "w")

    for i in range(len(pep)):
        ofile.write('>' + '\n' + pep[i] + '\n')
    ofile.close()

    # peptide list for parsing
    ofile = open(file_name + '_2.fasta', "w")

    for i in range(len(pep)):
        ofile.write(pep[i] + '\n')
    ofile.close()
    
    return

In [57]:
pwd

'c:\\Users\\rishy\\OneDrive\\KTH\\Thesis\\Immunopeptidomics'

In [ ]:
#make databse from known HLA peptides fasta file
!makeblastdb -in db/APD_Hs_all.fasta -dbtype prot -out db/APD_Hs_all

In [ ]:
#blast against canonical HLA peptides
!blastp -task blastp-short -query fasta.fasta -db db/APD_Hs_all -out HLA_blast_out -evalue 10.0 -outfmt "6 qseqid saccver pident qlen mismatch qstart qend sstart send evalue bitscore gaps qseq sseq"

In [ ]:
#parsing and catergorizing HLA_blast output
parse_categorize('db/APD_Hs_all.fasta', 'HLA_blast_out', 'fasta_2.fasta')

In [111]:
#known HLA
output_HLA = pd.read_table('categorized_HLA_blast_out')
known = output_HLA[output_HLA["blastp_category"] == 'match to known protein']
list = known["Query"] 
pep = list.to_list()

ofile = open("known_HLA.fasta", "w")

for i in range(len(pep)):
    ofile.write('>' + '\n' + pep[i] + '\n')
ofile.close()

known = output_HLA[output_HLA["blastp_category"] != 'match to known protein']
list = known["Query"] 
pep = list.to_list()
create_files(pep, 'to_blastp')

In [ ]:
#make blast database from fasta
!makeblastdb -in db/human_canonical.fasta  -dbtype prot -parse_seqids -out db/human_canonical

In [67]:
#blast all proteins against human canonical proteins
!blastp -task blastp-short -query to_blastp.fasta -db db/human_canonical -out blastp_out_human_canonical -evalue 10.0 -outfmt "6 qseqid sseqid pident qlen mismatch qstart qend sstart send evalue bitscore gaps qseq sseq"

^C


In [112]:
#fn for parsing and categorizing blast output
def parse_categorize(database_fasta, blast_out, fasta2): #requires inputs with file extension; fasta2 - file list without fasta header ">"

    input1= SeqIO.parse(database_fasta,"fasta") # blastp reference database
    seqdb={}
    for record in input1:
        seq=str(record.seq)
        if record.id not in seqdb:
            seqdb[record.id]=seq

    input2= open(blast_out,"r") # blastp output
    input3= open(fasta2,"r") # novel peptide tab txt
    output= open('categorized_' + blast_out,"w")


    blastout={}
    hits_dic={}
    for line in input2:
        row=line.strip().split("\t")
        qid=row[-2]
        sid=row[1]
        sseq=seqdb[sid]
        ident=row[2]
        peplen=int(row[3])
        mismatch=row[4]
        alignlen=int(row[6])-int(row[5])+1
        sstart=int(row[7])
        send=int(row[8])
        gap=row[11]
        evalue=float(row[-5])
        alignseq=row[-1]
        category="NA"
        single_sub_pos="NA"
        if sstart>3:
            Nterm_seq=sseq[sstart-4:sstart+2] #check up 3 amino acid before N-term of this peptide
        else:
            Nterm_seq=sseq[:sstart]

        if len(sseq)-send<3:
            Cterm_seq=sseq[send-1:]
        else:
            Cterm_seq=sseq[send-3:send+3]

        if alignlen==peplen:
            if float(ident)==100:
                category="match to known protein"
            
            elif int(gap)==0 and int(mismatch)==1:
                category="map to known protein with 1 aa mismatch"
                for i in range(peplen):
                    if qid[i]!=alignseq[i]:
                        single_sub_pos=str(i+1)

            elif int(gap)==1 and int(mismatch)==0:
                category="map to known protein with 1 aa insertion"
            else:
                category="novelpep (map to known protein with more than 2 mismatched aa)"
        elif peplen-alignlen==1 and float(ident)==100:
            category="map to known protein with 1 aa deletion"

        else:
            category="novelpep (map to known protein with more than 2 mismatched aa)"
        
        if qid not in hits_dic:
            hits_dic[qid]=evalue
            blastout[qid]=[category,sid,ident,peplen,single_sub_pos,Nterm_seq,alignseq,Cterm_seq,alignlen,mismatch,gap]
        else:
            if evalue<hits_dic[qid]:
                hits_dic[qid]=evalue
                blastout[qid]=[category,sid,ident,peplen,single_sub_pos,Nterm_seq,alignseq,Cterm_seq,alignlen,mismatch,gap]

    #header=input3.readline().strip().split("\t")

    header=["Query","blastp_category","blastp_match","identity","peplen","sub_pos","Nterm-seq(3aa)","aligned_seq","Cterm-seq(3aa)","alignlen","mismatch","gap"]
    output.write("\t".join(header)+"\n")

    for line in input3:
        row=line.strip().split("\t")
        peptide=row[0]
        if peptide in blastout:
            results=blastout[row[0]]
            newrow=row+results
            output.write("\t".join(map(str,newrow))+"\n")
        else:
            newrow=row+["novelpep (no match to known protein found)","NA","NA","NA","NA","NA","NA","NA","NA","NA","NA"]
            output.write("\t".join(map(str,newrow))+"\n")


    input2.close()
    input3.close()
    output.close()
   
    #adding headers to the novel blast output
    header_names = ["qseqid", "sseqid", "pident", "qlen", "mismatch", "qstart", "qend", "sstart", "send", "evalue", "bitscore", "gaps", "qseq", "sseq"]
    novel_file = pd.read_table('categorized_' + blast_out, sep ='\t', names = header_names)
    header_names = ["qseqid", "sseqid", "pident", "qlen", "mismatch", "qstart", "qend", "sstart", "send", "evalue", "bitscore", "gaps", "qseq", "sseq"]
    blast_file = pd.read_table(blast_out, sep ='\t', names = header_names)
    blast_file
    #to check if any keys are missing in the dictionary seqdb[]
    missing = blast_file[~blast_file["sseqid"].isin(seqdb.keys())]
    if (len(missing) == 0):
        return 'all is gud maan'

In [ ]:
count = 0 #checikng seqdb
for key, value in seqdb.items():
    print(key, ":", value)
    count += 1
    if count == 50:
        break

In [114]:
parse_categorize('db/human_canonical.fasta', 'blastp_out_human_canonical', 'to_blastp_2.fasta')

'all is gud maan'

In [117]:
#preparing fasta files of proteins with 1 AA mismatch
output  = pd.read_table('categorized_blastp_out_human_canonical')
mismatched = output[output["blastp_category"] == 'map to known protein with 1 aa mismatch']
list = mismatched["Query"] 
pep = list.to_list()

create_files(pep,'SAAV')

In [118]:
#preparing fasta files of proteins to search 6FT database
to_6ft = output[(output["blastp_category"] != 'map to known protein with 1 aa mismatch') & (output["blastp_category"] != 'match to known protein')]
list = to_6ft["Query"] 
pep = list.to_list()

create_files(pep, 'to_6ft')

In [125]:
#blastp against SAAV
!blastp -task blastp-short -query SAAV.fasta -db /mnt/d/Period 6/db/sap_db -out blastp_out_SAAV -evalue 10.0 -outfmt "6 qseqid saccver pident qlen mismatch qstart qend sstart send evalue bitscore gaps qseq sseq"

USAGE
  blastp [-h] [-help] [-import_search_strategy filename]
    [-export_search_strategy filename] [-task task_name] [-db database_name]
    [-dbsize num_letters] [-gilist filename] [-seqidlist filename]
    [-negative_gilist filename] [-negative_seqidlist filename]
    [-taxids taxids] [-negative_taxids taxids] [-taxidlist filename]
    [-negative_taxidlist filename] [-no_taxid_expansion] [-ipglist filename]
    [-negative_ipglist filename] [-entrez_query entrez_query]
    [-db_soft_mask filtering_algorithm] [-db_hard_mask filtering_algorithm]
    [-subject subject_input_file] [-subject_loc range] [-query input_file]
    [-out output_file] [-evalue evalue] [-word_size int_value]
    [-gapopen open_penalty] [-gapextend extend_penalty]
    [-qcov_hsp_perc float_value] [-max_hsps int_value]
    [-xdrop_ungap float_value] [-xdrop_gap float_value]
    [-xdrop_gap_final float_value] [-searchsp int_value] [-seg SEG_options]
    [-soft_masking soft_masking] [-matrix matrix_name]
    [-thre